# Agent-Based Optimization


In [1]:
import random

class Agent:
    def __init__(self, id, goals, behaviors):
        self.id = id
        self.goals = goals
        self.behaviors = behaviors

class ProductAgent(Agent):
    def __init__(self, id, product_name, quantity, due_date):
        super().__init__(id, ["maximize_production", "meet_due_date"], ["request_production"])
        self.product_name = product_name
        self.quantity = quantity
        self.due_date = due_date

    def request_production(self, scheduler):
        scheduler.receive_production_request(self)

class MachineAgent(Agent):
    def __init__(self, id, machine_name, capacity, availability):
        super().__init__(id, ["maximize_utilization", "minimize_downtime"], ["produce_product"])
        self.machine_name = machine_name
        self.capacity = capacity
        self.availability = availability
        self.production_schedule = []

    def produce_product(self, product_agent, scheduler):
        production_time = self.calculate_production_time(product_agent.quantity)
        self.production_schedule.append((product_agent, production_time))
        scheduler.allocate_resource(self, product_agent, production_time)

    def calculate_production_time(self, quantity):
        return round(max(1, quantity / self.capacity), 2)  # Ensure non-zero time rounded to two decimal places

class LaborAgent(Agent):
    def __init__(self, id, labor_name, availability):
        super().__init__(id, ["maximize_utilization", "minimize_idle_time"], ["perform_task"])
        self.labor_name = labor_name
        self.availability = availability
        self.task_schedule = []

    def perform_task(self, machine_agent, task_duration):
        self.task_schedule.append((machine_agent, task_duration))

class StorageAgent(Agent):
    def __init__(self, id, storage_name, capacity):
        super().__init__(id, ["minimize_inventory", "maximize_storage_utilization"], ["store_material"])
        self.storage_name = storage_name
        self.capacity = capacity
        self.inventory = 0

    def store_material(self, quantity):
        if self.inventory + quantity > self.capacity:
            raise Exception(f"Storage capacity exceeded in {self.storage_name}!")
        else:
            self.inventory += quantity

class SchedulerAgent(Agent):
    def __init__(self):
        super().__init__("scheduler", ["optimize_production_schedule"], ["allocate_resource"])
        self.production_requests = []
        self.resource_allocations = []

    def receive_production_request(self, product_agent):
        self.production_requests.append(product_agent)

    def allocate_resource(self, machine_agent, product_agent, production_time):
        self.resource_allocations.append((machine_agent, product_agent, production_time))

    def optimize_production_schedule(self):
        self.production_requests.sort(key=lambda x: x.due_date)  # Sort by due date for priority
        for product_agent in self.production_requests:
            machine_agent = self.select_machine(product_agent)
            if machine_agent:
                labor_agent = self.select_labor(machine_agent)
                if labor_agent:
                    storage_agent = self.select_storage(machine_agent, product_agent)
                    if storage_agent:
                        machine_agent.produce_product(product_agent, self)
                        labor_agent.perform_task(machine_agent, machine_agent.calculate_production_time(product_agent.quantity))
                        storage_agent.store_material(product_agent.quantity)
                    else:
                        print("No available storage.")
                else:
                    print("No available labor.")
            else:
                print("No available machine.")

    def select_machine(self, product_agent):
        # Select the machine with the lowest current workload
        if machine_agents:
            return min((m for m in machine_agents if m.availability), key=lambda x: sum(allocation[2] for allocation in self.resource_allocations if allocation[0] == x), default=None)
        return None

    def select_labor(self, machine_agent):
        available_labors = [l for l in labor_agents if l.availability]
        return random.choice(available_labors) if available_labors else None

    def select_storage(self, machine_agent, product_agent):
        available_storages = [s for s in storage_agents if s.capacity - s.inventory >= product_agent.quantity]
        return random.choice(available_storages) if available_storages else None

# Define agents
product_agents = [
    ProductAgent("P1", "Product A", 100, "2023-01-15"),
    ProductAgent("P2", "Product B", 150, "2023-01-20"),
    ProductAgent("P3", "Product C", 200, "2023-01-25")
]

machine_agents = [
    MachineAgent("M1", "Machine 1", 50, True),
    MachineAgent("M2", "Machine 2", 30, True),
    MachineAgent("M3", "Machine 3", 70, False)
]

labor_agents = [
    LaborAgent("L1", "Labor 1", True),
    LaborAgent("L2", "Labor 2", False)
]

storage_agents = [
    StorageAgent("S1", "Storage 1", 500),
    StorageAgent("S2", "Storage 2", 300)
]

scheduler_agent = SchedulerAgent()

# Initialize and run the simulation
for product_agent in product_agents:
    product_agent.request_production(scheduler_agent)

scheduler_agent.optimize_production_schedule()

# Print production schedules and resource allocations
print("Production Schedules and Machine Utilization:")
for allocation in scheduler_agent.resource_allocations:
    machine, product, time = allocation
    print(f"Machine {machine.machine_name} assigned to produce {product.product_name}, requiring {time} hours.")

print("\nStorage Capacities:")
for storage in storage_agents:
    print(f"Storage {storage.storage_name} has {storage.capacity - storage.inventory} units of capacity remaining, total capacity: {storage.capacity}")

print("\nMachine Availability:")
for machine in machine_agents:
    print(f"Machine {machine.machine_name} is {'available' if machine.availability else 'not available'}, Total Production Scheduled: {sum(allocation[2] for allocation in scheduler_agent.resource_allocations if allocation[0] == machine)} hours")


Production Schedules and Machine Utilization:
Machine Machine 1 assigned to produce Product A, requiring 2.0 hours.
Machine Machine 2 assigned to produce Product B, requiring 5.0 hours.
Machine Machine 1 assigned to produce Product C, requiring 4.0 hours.

Storage Capacities:
Storage Storage 1 has 200 units of capacity remaining, total capacity: 500
Storage Storage 2 has 150 units of capacity remaining, total capacity: 300

Machine Availability:
Machine Machine 1 is available, Total Production Scheduled: 6.0 hours
Machine Machine 2 is available, Total Production Scheduled: 5.0 hours
Machine Machine 3 is not available, Total Production Scheduled: 0 hours
